In [10]:

import json
import requests
from langchain_community.llms import Ollama

json_file = "https://clchatagentassessment.s3.ap-south-1.amazonaws.com/queries.json"
output_file = "output.json"

llm = Ollama(model="llama3.1")

In [11]:
!ollama pull llama3.1

pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 11ce4ee3e170... 100% ▕████████████████▏ 1.7 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 1a4c3c319823... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [12]:
llm = Ollama(model="llama3.1")
query="name the capital of india"
llm.invoke(query)

'The capital of India is **New Delhi**.'

In [20]:
def summarize_diet_chart_enhanced(diet_chart):
    summary = {}
    
    summary["general_notes"] = diet_chart.get('notes', "")
    
    current_day = diet_chart['meals_by_days'][0]
    meals_summary = {}
    
    for meal in current_day['meals']:
        meal_name = meal.get('name', "Unnamed Meal")
        meal_timing = meal.get('timings', "No Time Specified")
        
        meal_info = {
            "timing": meal_timing,
            "food_items": [],
            "meal_notes": []
        }
        
        for option in meal.get('meal_options', []):
            meal_info["meal_notes"].append(option.get('notes', ""))
            
            food_items = [food_item['Food'].get('name', "Unknown Food Item") 
                          for food_item in option.get('meal_option_food_items', [])]
            meal_info["food_items"].extend(food_items)
        
        meals_summary[meal_name] = meal_info
    
    summary["meals"] = meals_summary
    
    print(json.dumps(summary, indent=2))
    
    return summary

In [13]:
def summarize_chat_history(chat_history):
    return chat_history[-5:]

In [20]:
def write_output(output):
    with open(output_file, "w") as f:
        json.dump(output, f, indent=2)
    print(f"Output written to {output_file}")

In [14]:
def fetch_queries():
    response = requests.get(json_file)
    return response.json()

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest 
Error: pull model manifest: file does not exist


In [22]:
def generate_response(query):
    profile_context = query.get("profile_context", {})
    latest_query = query.get("latest_query", {})
    chat_context = query.get("chat_context", {})

    # Handle missing 'diet_chart' gracefully
    diet_chart = profile_context.get('diet_chart', {})
    summarized_diet_chart = summarize_diet_chart_enhanced(diet_chart) if diet_chart else {}
    summarized_chat_history = summarize_chat_history(chat_context.get('chat_history', []))

    prompt = f"""
        Patient Profile:
        {profile_context.get('patient_profile', {})}

        Program: {profile_context.get('program_name', '')}

        General Notes from Diet Chart:
        {summarized_diet_chart.get('general_notes', '')}

        Today's Diet Chart:
    """

    for meal_name, meal_info in summarized_diet_chart.get("meals", {}).items():
        prompt += f"\nMeal: {meal_name} ({meal_info.get('timing', 'No Time Specified')})\n"
        prompt += "Foods:\n"
        for food in meal_info.get('food_items', []):
            prompt += f"- {food}\n"
        if meal_info.get('meal_notes', []):
            prompt += "Meal-specific Notes:\n"
            for note in meal_info['meal_notes']:
                prompt += f"- {note}\n"

    prompt += f"""

    Latest Query:
    {json.dumps(latest_query, indent=2)}

    Recent Chat History:
    {json.dumps(summarized_chat_history, indent=2)}

    Based on the above information, provide a concise and actionable response to the patient's latest query. 
    Follow these guidelines:
    1. Compare the meal in the query to the ideal meal from the diet chart for that specific time.
    2. Check for compliance, noting any missing items or extra items consumed by the patient.
    3. Consider any details from the image description, including the caption provided in the text.
    4. Provide specific advice based on the patient's medical profile and recent chat history.
    5. Use the same messaging style and tone as the patient, considering the chat history for guidance. 
    6. Keep the response concise and actionable.
    7. Dont start the messages by "based on the diet chart" and other such phrases, it feels like response is AI generated. 
    8. Talk to the patient like you are their family doctor, i mean you should respond in the way the have asked(you can use hindi also(typed using english letters)). Your tone and messaging style should be the same as them.
    9. At the end of each response, you might add a daily reminder taking reference from the general notes for the patient and suggest them a healthy thing to do which is mentioned in the general notes(dont say like- "as per your general notes', just sound like a human)
    """

    response = llm.generate(prompts=[prompt])
    
    if response.generations and response.generations[0]:
        generated_text = response.generations[0][0].text
        return generated_text.strip()
    else:
        return "No response generated."




In [23]:
def main():
    url = "https://clchatagentassessment.s3.ap-south-1.amazonaws.com/queries.json"
    
    ideal_responses = fetch_all_ideal_responses(url)
    latest_queries = fetch_latest_queries(url)
    ticket_ids = fetch_all_ticket_ids(url)
    
    if len(ticket_ids) != len(latest_queries) or len(latest_queries) != len(ideal_responses):
        raise ValueError("Mismatch in the number of ticket_ids, latest_queries, and ideal_responses.")
    
    queries = [{"profile_context": {}, "latest_query": query, "chat_context": {"ticket_id": ticket_id}}
               for query, ticket_id in zip(latest_queries, ticket_ids)]
    
    generated_responses = [generate_response(query) for query in queries]
    
    output_data = []
    for ticket_id, latest_query, generated_response, ideal_response in zip(ticket_ids, latest_queries, generated_responses, ideal_responses):
        output_data.append({
            "ticket_id": ticket_id,
            "latest_query": latest_query,
            "generated_response": generated_response,
            "ideal_response": ideal_response
        })
    
    with open('output.json', 'w') as f:
        json.dump(output_data, f, indent=2)
    
    print("Output file 'output.json' has been created.")

if __name__ == "__main__":
    main()

Output file 'output.json' has been created.


In [66]:

def fetch_ideal_response(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        summary = {}

        if isinstance(data, list):
            if len(data) > 0 and isinstance(data[0], dict):
                ideal_response = data[0].get('ideal_response', "No Ideal Response Provided")
            else:
                print("The list does not contain a valid dictionary.")
                return None
        elif isinstance(data, dict):
            
            ideal_response = data.get('ideal_response', "No Ideal Response Provided")
        else:
            print("Unexpected data format.")
            return None
        
        summary["ideal_response"] = ideal_response
        
        return summary
    else:
        print("Failed to retrieve data. Status code:", response.status_code)
        return None

url = "https://clchatagentassessment.s3.ap-south-1.amazonaws.com/queries.json"
result = fetch_ideal_response(url)
if result:
    print(json.dumps(result, indent=2))




{
  "ideal_response": "Great job for having methi water, continue having it daily, it will help boost your metabolism.\nVarsha, but i also noticed that you are having figs and raisins but they are not presrcibed in the diet plan, can i know why you have added them ?"
}


In [18]:

def fetch_all_ticket_ids(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        ticket_ids = []
        
        if isinstance(data, list):
            for item in data:
                if isinstance(item, dict):
                    chat_context = item.get('chat_context', {})
                    
                    ticket_id = chat_context.get('ticket_id', None)
                    
                    if ticket_id:
                        ticket_ids.append(ticket_id)
        elif isinstance(data, dict):
            chat_context = data.get('chat_context', {})
            ticket_id = chat_context.get('ticket_id', None)
            if ticket_id:
                ticket_ids.append(ticket_id)
        else:
            print("Unexpected data format.")
            return None
        
        return ticket_ids
    else:
        print("Failed to retrieve data. Status code:", response.status_code)
        return None

url = "https://clchatagentassessment.s3.ap-south-1.amazonaws.com/queries.json"
ticket_ids = fetch_all_ticket_ids(url)
if ticket_ids:
    print(json.dumps(ticket_ids, indent=2))



[
  "3a16cf52-26a0-47d6-8c91-9810039676ee",
  "3a078f8d-b2b9-41af-a592-5f1c6cdd0484",
  "8e342394-8e9f-42d5-9905-52d570fa9c2f",
  "4937b689-8013-4316-80ec-e66447c387c3",
  "144d6db8-d7ad-482b-9de3-540bf39127f5",
  "8a280b8a-26ab-40f5-bb1d-22b1aa4e47f3",
  "15d414f1-3afc-4b4b-adc6-fafc3eaf3f8f",
  "158e30e7-eb1b-4986-8515-05180028d296",
  "1bc93246-b66f-409f-bbc1-dfec4cd2ae8d",
  "e2ef7155-002b-4bdf-a8ef-270a8c59b025",
  "ca331178-80da-48f8-a3a5-d483fa0391ea",
  "ffa2b043-7c9f-43e3-b434-7f874f01b916",
  "206b8b1e-25f6-48fb-9271-524f1752f27c",
  "cb61d429-1350-41e0-b312-e83a01be382b",
  "699bb2fa-0192-47df-ae49-dafd50c42238",
  "53a39079-04ce-440f-8fe6-4a6d1e6a570d",
  "caf28ae3-7327-4f61-ae15-2c18597d8e3e",
  "9f991db6-ed10-4613-b597-040ae4cf6d92",
  "d9cc7206-f263-4e5f-a095-f3dbb12d3fcc",
  "7bcaa083-5f9e-4142-9976-9172ed6e5c90",
  "84447c63-2773-488c-a799-c1702c200bff",
  "42eed415-aee2-4ced-8772-89c086b779dc",
  "05c707a6-aa65-4411-be6a-834af5df2ddd"
]


In [16]:

def fetch_all_ideal_responses(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        ideal_responses = []
        
        if isinstance(data, list):
            for item in data:
                if isinstance(item, dict):
                    ideal_response = item.get('ideal_response', None)
                    
                    if ideal_response:
                        ideal_responses.append(ideal_response)
        elif isinstance(data, dict):
            ideal_response = data.get('ideal_response', None)
            if ideal_response:
                ideal_responses.append(ideal_response)
        else:
            print("Unexpected data format.")
            return None
        
        return ideal_responses
    else:
        print("Failed to retrieve data. Status code:", response.status_code)
        return None


url = "https://clchatagentassessment.s3.ap-south-1.amazonaws.com/queries.json"
ideal_responses = fetch_all_ideal_responses(url)
if ideal_responses:
    print(json.dumps(ideal_responses, indent=2))


[
  "Great job for having methi water, continue having it daily, it will help boost your metabolism.\nVarsha, but i also noticed that you are having figs and raisins but they are not presrcibed in the diet plan, can i know why you have added them ?",
  "Varsha, i noticed you are having upma instead of poha as prescribed in your diet plan, can you let me know why ?",
  "Shobhita, I noticed aap oats le rhe ho, which is a healthy choice, but maine aapki diet mein aloo parantha + curd likha tha, aapne vo nhi khaya ?",
  "Great job on following your diet plan, keep it up!",
  "Great job sticking to your meal plan! Keep it up!",
  "Great job aarti, why have you not included milk ?",
  "Great job, keep it up! Sesame and sunflower seeds are great source of omega 3 fatty acid which help regulate your period cycle.",
  "Good job, coconut water is great source of potassium. Why have you not included seeds with this ?",
  "Rupa, I noticed that you were prescribed mixed veg vermicili, can you let k

In [17]:
def fetch_latest_queries(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        latest_queries = [entry.get('latest_query', {}) for entry in data]
        return latest_queries
    else:
        raise Exception(f"Failed to retrieve latest queries. Status code: {response.status_code}")
fetch_latest_queries(url)



[[{'role': 'user',
   'content': 'Sent an image (The image shows a hand holding a small metal bowl containing yellowish-brown seeds soaked in water. The background includes a bed with a colorful bedsheet and a tiled wall.)'},
  {'role': 'user',
   'content': "Sent an image (The image shows a small metal bowl containing a variety of dried fruits and nuts. Visible items include a dried fig, almonds, walnuts, and raisins. The bowl is placed on a dark fabric surface, possibly someone's lap.) with the following caption Dry fruits."},
  {'role': 'user',
   'content': 'Replied to the following message: "Sent an image (The image shows a hand holding a small metal bowl containing yellowish-brown seeds soaked in water. The background includes a bed with a colorful bedsheet and a tiled wall.)", with: "Methi dana done"'}],
 [{'role': 'user',
   'content': 'Sent an image (The image shows a bowl of a yellow-orange colored dish, likely a type of porridge or savory pudding, with visible black mustard 